# Spectral element DEComposition (SDEC) Plot
The SDEC Plot illustrates the contributions of different chemical elements in the formation of a simulation model's spectrum. It is a spectral diagnostic plot similar to those originally proposed by M. Kromer (see, for example, [Kromer et al. 2013](https://arxiv.org/abs/1311.0310), figure 4).

In [ ]:
# We filter out warnings throughout this notebook
import warnings
warnings.filterwarnings('ignore');

First, create and run a simulation for which you want to generate this plot:

In [ ]:
from tardis import run_tardis
from tardis.io.atom_data.util import download_atom_data

# We download the atomic data needed to run the simulation
download_atom_data('kurucz_cd23_chianti_H_He')

sim = run_tardis("tardis_example.yml", virtual_packet_logging=True)

<div class="alert alert-info">

Note

The virtual packet logging capability must be active in order to produce the SDEC Plot for virtual packets population. Thus, make sure to set `virtual_packet_logging: True` in your configuration file if you want to generate the SDEC Plot with virtual packets. It should be added under the `virtual` property of the `spectrum` property, as described in the [configuration schema](https://tardis-sn.github.io/tardis/io/configuration/components/spectrum.html).

</div>

Now, import the plotting interface for the SDEC plot, i.e. the `SDECPlotter` class.

In [ ]:
from tardis.visualization import SDECPlotter

And create a plotter object to process the data of simulation object `sim` for generating the SDEC plots.

In [ ]:
plotter = SDECPlotter.from_simulation(sim)

## Static Plot (in matplotlib)
You can now call the `generate_plot_mpl()` method on your `plotter` object to get a highly informative, yet beautiful, SDEC plot produced in matplotlib.

### Virtual packets mode
By default, an SDEC plot is produced for the virtual packet population of the simulation.

In [ ]:
plotter.generate_plot_mpl();

### Real packets mode
You can produce the SDEC plot for the real packet population of the simulation by setting `packets_mode="real"` which is `"virtual"` by default. Since `packets_mode` is the 1st argument, you can simply pass `"real"` string only.

In [ ]:
plotter.generate_plot_mpl("real");

### Plotting a specific wavelength range
You can also restrict the wavelength range of escaped packets that you want to plot by specifying `packet_wvl_range`. It should be a quantity in Angstroms, containing two values - lower lambda and upper lambda i.e. `[lower_lambda, upper_lambda] * u.AA`.

In [ ]:
from astropy import units as u

In [ ]:
plotter.generate_plot_mpl(packet_wvl_range=[3000, 9000] * u.AA);

### Plotting only the top contributing elements

The `nelements` option allows you to plot the top contributing elements to the spectrum. The top elements are shown in unique colors and the rest of the elements are shown in silver. Please note this works only for elements and not for ions.

In [ ]:
plotter.generate_plot_mpl(packet_wvl_range=[2000, 8000] * u.AA, nelements = 3);

### Choosing what elements/ions to plot

You can also pass a list of elements/ions of your choice in the `species_list` option and plot them. Valid options include elements (e.g. Si), ions (which must be specified in Roman numeral format, e.g. Si II), a range of ions (e.g. Si I-III), or any combination of these.

In [ ]:
plotter.generate_plot_mpl(packet_wvl_range=[2000, 8000] * u.AA, species_list = ['Si II', 'S I-V', 'Ca']);

When using both the `nelements` and the `species_list` options, `species_list` takes precedence. 

In [ ]:
plotter.generate_plot_mpl(nelements = 3, species_list = ['Si II', 'S I-V', 'Ca']);

### Plotting flux instead of luminosity
You can plot in units of flux on the Y-axis of the SDEC plot, by specifying the `distance` parameter. It should be a quantity with a unit of length like m, Mpc, etc. and must be a positive value. By default, `distance=None` plots luminosity on the Y-axis.

In [ ]:
plotter.generate_plot_mpl(distance=100 * u.Mpc);

### Plotting an observed spectrum
To add an observed spectrum to the SDEC plot, you would need to pass the wavelength and the flux to the `observed_spectrum` parameter. The argument passed should be a tuple/list where the first value is the wavelength and the second value is the flux of the observed spectrum. Please note that these values should be instances of `astropy.Quantity`. 

In [ ]:
import numpy as np
data = np.loadtxt('demo_observed_spectrum.dat')

observed_spectrum_wavelength, observed_spectrum_flux = data.T
observed_spectrum_wavelength = observed_spectrum_wavelength * u.AA
observed_spectrum_flux = observed_spectrum_flux *   u.erg / (u.s * u.cm**2 * u.AA)

In [ ]:
plotter.generate_plot_mpl(observed_spectrum = (observed_spectrum_wavelength, observed_spectrum_flux), distance = 6 * u.Mpc);

### Hiding modeled spectrum
By default, the modeled spectrum is shown in SDEC plot. You can hide it by setting `show_modeled_spectrum=False`.

In [ ]:
plotter.generate_plot_mpl(show_modeled_spectrum=False);

### Additional plotting options

In [ ]:
# To list all avaialble options (or parameters) with their description
help(plotter.generate_plot_mpl)

The `generate_plot_mpl` method also has options specific to the matplotlib API, thereby providing you with more control over how your SDEC plot looks. Possible cases where you may use them are:

- `ax`: To plot SDEC on the Axis of a plot you're already working with, e.g. for subplots.

- `figsize`: To resize the SDEC plot as per your requirements.

- `cmapname`: To use a colormap of your preference, instead of "jet".

## Interactive Plot (in plotly)
If you're using the SDEC plot for exploration purposes, you should plot its interactive version by using `generate_plot_ply()`. This not only allows you to zoom & pan but also to inspect data values by hovering, to resize scale, etc. conveniently (as shown below).

![Interactions possible with SDEC plotly plot](../images/sdec_ply_interaction.gif)


**This method takes the exact same arguments as `generate_plot_mpl` except a few that are specific to the plotting library.** We can produce all the plots above in plotly, by passing the same arguments.

### Virtual packets mode

In [ ]:
plotter.generate_plot_ply()

### Real packets mode

In [ ]:
plotter.generate_plot_ply(packets_mode="real")

### Plotting a specific wavelength range

In [ ]:
plotter.generate_plot_ply(packet_wvl_range=[3000, 9000] * u.AA)

### Plotting only the top contributing elements

In [ ]:
plotter.generate_plot_ply(packet_wvl_range=[2000, 8000] * u.AA, nelements = 3)

### Choosing what elements/ions to plot

In [ ]:
plotter.generate_plot_ply(packet_wvl_range=[2000, 8000] * u.AA, species_list = ['Si II', 'S I-V', 'Ca'])

### Plotting an observed spectrum

In [ ]:
plotter.generate_plot_ply(observed_spectrum = (observed_spectrum_wavelength, observed_spectrum_flux), distance = 6 * u.Mpc)

In a similar manner, you can also use `show_modeled_spectrum` and `distance` argument in plotly plots.

### Additional plotting options

The `generate_plot_ply` method also has options specific to the plotly API, thereby providing you with more control over how your SDEC plot looks. Possible cases where you may use them are:
- `fig`: To plot the SDEC plot on a figure you are already using e.g. for subplots.
- `graph_height`: To specify the height of the graph as needed.
- `cmapname`: To use a colormap of your preference instead of "jet".

In [ ]:
# To list all avaialble options (or parameters) with their description
help(plotter.generate_plot_ply)

## Using simulation saved as HDF
Other than producing the SDEC Plot for simulation objects in runtime, you can also produce it for saved TARDIS simulations.

In [ ]:
hdf_plotter = SDECPlotter.from_hdf("demo.hdf")

This `hdf_plotter` object is similar to the `plotter` object we used above, **so you can use each plotting method demonstrated above with this too.**

In [ ]:
# Static plot with virtual packets mode
hdf_plotter.generate_plot_mpl();

In [ ]:
# Static plot with real packets mode
hdf_plotter.generate_plot_mpl("real");

In [ ]:
# Interactive plot with virtual packets mode
hdf_plotter.generate_plot_ply()